# Лекция 6: Работа с базами данных

__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2020 г. 

При подготовке лекции использованы материалы:
* ...

V 0.2 07.10.2020

In [1]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v1.css")
HTML(html.read().decode('utf-8'))

## Разделы: <a class="anchor" id="разделы"></a>

* [SQLite](#sqlite)
* [Нормальные формы](#нормальные-формы) 

-
* [к оглавлению](#разделы)

## SQLite <a class="anchor" id="sqlite"></a>
-
* [к оглавлению](#разделы)

__SQLite__ — компактная встраиваемая СУБД. SQLite реализована на языке C. SQLite представляет собой SQL-движок со следующими фичами:
* компактный
* быстрый
* автономный
* надежный
* полнофункциональный

* "Встраиваемая" (embedded) означает, что SQLite не использует парадигму клиент-сервер. Т.е. движок SQLite
    * __НЕ является отдельно работающим процессом__, с которым взаимодействует программа;
    * __представляет собой библиотеку__, с которой программа компонуется, и движок становится составной частью программы.
    * обменивается данными с клиентским кодом с использованием вызовов функций (API) библиотеки SQLite, что уменьшает накладные расходы, время отклика и упрощает программу. 
* SQLite хранит всю базу данных (включая определения, таблицы, индексы и данные) в единственном стандартном файле на том компьютере, на котором исполняется программа. 
 
* Перед началом исполнения транзакции записи весь файл, хранящий базу данных, блокируется, что обеспечивает простоту их реализации. 
* ACID-функции достигаются в том числе за счёт создания файла журнала.


__SQLite - самая распространенная СУБД в мире__. SQLite встроена во все современные мобильные телефоны и в большинство компьютеров и в бесчисленное множество других приложений. По умолчанию SQLite включена в:

* Google's Android
* LG's webOS
* Windows 10
* Apple adopted it as an option in macOS's Core Data API from the original implementation in Mac OS X 10.4 onwards, and also for administration of videos and songs, and in iOS for storage of text messages on the iPhone.[43]


* Blackberry's BlackBerry 10 OS
* Symbian OS
* Nokia's Maemo
* Linux Foundation's MeeGo
* NetBSD
* FreeBSD where starting with 10-RELEASE version in January 2014, it is used by the core package management system.
illumos
* Oracle Solaris 10 where the Service Management Facility database is serialized for booting.
* MorphOS since version 3.10
* Tizen


## Доступ к базе данных SQLite из Python

In [24]:
import  sqlite3 # Подключаем модуль 
sqlite3.apilevel #  Получаем номер  сnецификации 

'2.0'

In [25]:
sqlite3.sqlite_version

'3.30.0'

Согласно спецификации DB-API  2.0  последовательность работы с  базой данных выглядит 
следующим образом: 

1. Производится __подключение__ к базе данных с помощью функции `connect()`. Функция возвращает объект соединения, с  помощью которого осуществляется дальнейшая работа с базой данных. 
2. Создается __объект-курсор__.
3. __Выnолняются SQL-зanpocы и обрабатываются результаты__. 
    Перед выnолнением nервого заnроса. который изменяет заnиси (INSERT,  REPIACE,  UPDATE  и DELETE),  автоматически заnускается транзакция. 
4. __Завершается транзакция__ или отменяются все изменения в рамках транзакции. 
5. __Закрывается объект-курсор__. 
6. __Закрывается соединение__ с базой данных. 

### Создание и открытие базы данных

Дпя создания  и  открытия базы данных исnользуется  функция `connect()`.  Функция имеет следующий формат:

`connect(database[,  timeout] [,  lsolatlon_level] [,  detect  types] [,  factory]  [,  check_same_thread] [,  cached_statements])`

* В nараметре database указывается абсолютный или относительный nуть к базе данных.
    * Если база данных не существует, то она будет создана и открыта для работы.
    * Если база данных уже существует, то она nросто открывается без удаления имеющихся данных. 
    * Вместо nути к базе данных можно указать значение  `:memory:`,  которое означает, что база данных будет создана в оnеративной nамяти. После закрытия такой базы все данные будут удалены. 

* Все __остальные nараметры являются необязательными__ и могут быть указаны в nроизвольнам порядке путем присвоения значения названию  параметра. 
* Необязательный nараметр `timeout` задает время ожидания снятия блокировки с открываемой базы данных. По умолчанию значение nараметра timeout  равно nяти секундам. 
Предназначение остальных nараметров мы рассмотрим немного nозже. 

Функция `connect()` возвращает объект соединения, с  nомощью которого осуществляется вся дальнейшая работа с базой данных. Если открыть базу данных не удалось, то возбуждается исключение. Соединение закрывается, когда вызывается метод `close()`  объекта соединения.

In [26]:
# import  sqliteЗ #  Подключаем модуль  sqliteЗ 
con = sqlite3.connect("testdb.db") # Открьrnаем базу данных 

#  Работаем с  базой данных

con.close() #  Закрьrnаем базу данных

После создания объекта соединения необходимо создать __объект-курсор__.  Все дальнейшие заnросы  должны  nроизводиться  через  этот  объект. Создание объекта-курсора nроизводится с  nомощью метода `cursor()`.  Для выnолнения заnроса к базе данных nредназначены следующие методы объекта-курсора: 

* `close()` - закрывает объект-курсор.
* `executescript(< SQL-зanpocы  через  точку  с  запятой>)` - выnолняет  несколько  SQL-зanpocoв за один раз. Если в nроцессе выnолнения заnросов возникает ошибка, то метод возбуждает исключение.
* `execute(< SQL-зaпpoc> [,  <Значения>])` - выполняет один SQL-зaпpoc. Если в  процессе выполнения  запроса  возникает  ошибка,  то  метод  возбуждает  исключение. 

In [27]:
# import  sqlite3

con = sqlite3.connect("catalog.db") 
cur = con.cursor() #  Создаем объект-курсор 

In [28]:
# несколько SQL выражений создающих простую БД:
sql  = """
CREATE TABLE ALBUM
(
    ALBUM_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
    TITLE VARCHAR(200)  NOT NULL,
    ARTIST_ID INTEGER  NOT NULL,
    FOREIGN KEY (ARTIST_ID) REFERENCES ARTIST (ARTIST_ID) 
);

CREATE TABLE ARTIST
(
    ARTIST_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    NAME VARCHAR(200),
    COMMENT VARCHAR(400)
);

CREATE TABLE GENRE
(
    GENRE_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    NAME VARCHAR(200)
);

CREATE TABLE PLAY_LIST
(
    PLAY_LIST_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    NAME VARCHAR(200)
);

CREATE TABLE PLAY_LIST_TRACK
(
    PLAY_LIST_ID INTEGER  NOT NULL,
    TRACK_ID INTEGER  NOT NULL,
    CONSTRAINT PLAY_LIST_TRACK PRIMARY KEY  (PLAY_LIST_ID, TRACK_ID),
    FOREIGN KEY (PLAY_LIST_ID) REFERENCES PLAY_LIST (PLAY_LIST_ID),
    FOREIGN KEY (TRACK_ID) REFERENCES TRACK (TRACK_ID) 
);

CREATE TABLE TRACK
(
    TRACK_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    NAME VARCHAR(200)  NOT NULL,
    ALBUM_ID INTEGER,
    MEDIA_TYPE_ID INTEGER  NOT NULL,
    GENRE_ID INTEGER,
    COMPOSER VARCHAR(200),
    MILLISECONDS INTEGER  NOT NULL,
    FOREIGN KEY (ALBUM_ID) REFERENCES ALBUM (ALBUM_ID),
    FOREIGN KEY (GENRE_ID) REFERENCES GENRE (GENRE_ID)
);""" 

In [29]:
try:  # Обрабатываем искточения 
    cur.executescript(sql)  # Выnолняем SQL-зanpocы 
except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    cur.close() # Закрываем объект-курсор 
    con.close() # Закрываем соединение
#     input() 

Запрос успешно выполнен


In [6]:
ls

 ’®¬ ў гбва®©бвўҐ C ­Ґ Ё¬ҐҐв ¬ҐвЄЁ.
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : D65B-CBD9

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Users\‘ҐаЈҐ©\YandexDisk\Python\Ipynb\TOBD_2020\06_database

08.10.2020  16:10    <DIR>          .
08.10.2020  16:10    <DIR>          ..
07.10.2020  16:03    <DIR>          .ipynb_checkpoints
08.10.2020  15:24         2я214я400 648989.ppt
07.10.2020  16:35            36я864 catalog.db
08.10.2020  15:11    <DIR>          img
06.10.2020  19:24             2я745 lec_v1.css
07.10.2020  17:05    <DIR>          materials
07.10.2020  15:56                 0 testdb.db
08.10.2020  16:10            74я413 TOBD_lec_06_database_v2..ipynb
06.10.2020  19:26             3я070 Untitled.ipynb
08.10.2020  15:24         2я226я176 ‚ўҐ¤Ґ­ЁҐ ў п§лЄ SQL.ppt
               7 д ©«®ў      4я557я668 Ў ©в
               5 Ї Ї®Є  35я729я932я288 Ў ©в бў®Ў®¤­®


__Добавление записи в таблицу__

В этом примере мы использовали метод `commit()` объекта соединения. 
* Метод `commit()` позволяет __завершить транзакцию__,  которая  запускается  автоматически.
* Если метод __не вызвать__ и при этом закрыть соединение с базой данных, то все произведенные изменения __будут автоматически отменены__. 
* Запросы, изменяющие записи (INSERT,  REPLACE,  UPDAТE, DELETE), __нужно завершать__ вызовом метода `commit()`.

In [30]:
# import  sqlite3
con = sqlite3.connect("catalog.db") 
cur = con.cursor() #  Создаем объект-курсор 

# SQL выражение 
sql  = """
INSERT INTO GENRE (NAME) 
VALUES ('Jazz')"""

In [31]:
try:  # Обрабатываем искточения 
    cur.execute(sql)  # Выnолняем SQL-зanpocы 
except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() # Завершаем транзакцию
    cur.close() # Закрываем объект-курсор 
    con.close() # Закрываем соединение
#     input() 

Запрос успешно выполнен


Часто в SQL-зaпpoc необходимо подставлять данные,  полученные от пользователя.  

* <b class="r">!!!</b> Если данные от пользователя не обработать и подставить в SQL-зaпpoc, то пользователь получает возможность видоизменить запрос и, например, зайти в закрытый раздел без ввода пароля. Такой вид атаки называется __SQL injection__.
* Чтобы значения были правильно подставлены, нужно их передавать в виде кортежа или словаря во втором параметре метода `execute()`.  В этом случае в SQL-зaпpoce указываются следующие специальные заполнители: 
    * `?` - при указании значения в виде кортежа; 
    * `:<Ключ>` - при указании значения в виде словаря.

In [32]:
# import  sqlite3
con = sqlite3.connect("catalog.db") 
cur = con.cursor() #  Создаем объект-курсор 
# SQL выражение 

In [33]:
# подстановка позиционного параметра:

sql_1 = '''
INSERT INTO Artist (Name) 
VALUES (?)'''
v1 = ('AC/DC',)
v2 = ('Aerosmith',)

In [34]:
# подстановка позиционного параметра (альтернативный синтаксис):

sql_2 = '''
INSERT INTO Artist (Name) 
VALUES ({})'''
v3 = "'Alanis Morissette'"

In [35]:
# подстановка нескольких позиционных параметров:

sql_3 = '''
INSERT INTO Artist (Name, Comment) 
VALUES (?, ?)'''
v4 = ('Alanis Morissette', 'My test comment')
v5 = ('Apocalyptica', 'It is my favorite artist')

In [36]:
# подстановка по ключу:

sql_4 = '''
INSERT INTO Artist (Name, Comment) 
VALUES (:n, :com)'''
v6 = {'n': "Guns N' Roses", 'com': 'Funny guys'}
v7 = {'n': 'Iron Maiden', 'com': """Test symbols v " v ' v """}

In [37]:
try:  
    cur.execute(sql_1, v1)
    cur.execute(sql_1, v2)
    
    # Опасный способ, если пользователь может передать значение:
    cur.execute(sql_2.format(v3))
    
    cur.execute(sql_3, v4)
    cur.execute(sql_3, v5)
    
    cur.execute(sql_4, v6)
    cur.execute(sql_4, v7) # тестирование экранирования символов
        
except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() # Завершаем транзакцию
    cur.close() # Закрываем объект-курсор 
    con.close() # Закрываем соединение

Запрос успешно выполнен


`executemany(< SQL-зaпpoc>, < Последовательность>)` - выполняет SQL-зaпpoc несколько раз,  при этом подставляя значения из последовательности.
* Каждый элемент последовательности должен быть кортежем (при  использовании заполнителя  `?`)  или словарем (при использовании заполнителя `:<Ключ>`).
* Вместо последовательности можно указать объект-итератор или объект-генератор. 
* Если в  процессе выполнения запроса возникает ошибка, то метод возбуждает исключение; 

In [38]:
# import  sqlite3
con = sqlite3.connect("catalog.db") 
cur = con.cursor() #  Создаем объект-курсор 
# SQL выражение 

sql = '''
INSERT INTO Artist (Name) 
VALUES (?)'''

val_list = [('The Clash',), ('The Cult',), ('The Doors',)]

try:  
    cur.executemany(sql, val_list)        
except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() # Завершаем транзакцию
    cur.close() # Закрываем объект-курсор 
    con.close() # Закрываем соединение

Запрос успешно выполнен


Модуль `sqlite3` содержит также методы `execute()`,  `executernany()`  и `executescript()` объекта соединения, которые позволяют выполнить запрос без создания объекта-курсора. 

In [39]:
# import  sqlite3
con = sqlite3.connect("catalog.db") 
# cur = con.cursor() #  НЕ создаем объект-курсор 
# SQL выражение 

sql = '''
INSERT INTO Artist (Name) 
VALUES (?)'''

val_list = [('The Police',), ('The Rolling Stones',)]

try:  
    con.executemany(sql, val_list)        
except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() # Завершаем транзакцию
    # cur.close() # Закрываем объект-курсор 
    con.close() # Закрываем соединение

Запрос успешно выполнен


Объект-курсор nоддерживает несколько атрибутов: 
* `lastrowid` - индекс  последней добавленной заnиси  с  nомощью инструкции  `INSERТ` и метода `execute()`.  Если индекс не определен, то атрибут будет содержать значение `None`. 

В качестве nримера добавим новую рубрику и выведем ее индекс: 
* `rowcount` - количество измененных или удаленных заnисей.  Если количество не оnределено, то атрибут имеет значение `-1`; 
* `description` - содержит кортеж кортежей с  именами nолей в  результате  выnолнения инструкции  `SELECT`. Каждый внутренний  кортеж состоит  из  семи элементов.  Первый элемент содержит название  nоля,  а  остальные элементы всегда  имеют значение `None`.

In [40]:
# import  sqlite3
con = sqlite3.connect("catalog.db") 
# cur = con.cursor() #  НЕ создаем объект-курсор 
# SQL выражение 

sql = '''
INSERT INTO Artist (Name) 
VALUES (?)'''

val_list = [('The Police',), ('The Rolling Stones',)]
val = ('The Tea Party',)

In [41]:
try:  
    res = con.executemany(sql, val_list)        
    print('rowcount:', res.rowcount)
    print('descriptiont:', res.description)    
    
    res = con.execute(sql, val)        
    print('lastrowid:', res.lastrowid)
    
except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() # Завершаем транзакцию
    # cur.close() # Закрываем объект-курсор 
    con.close() # Закрываем соединение

rowcount: 2
descriptiont: None
lastrowid: 15
Запрос успешно выполнен


###  Обработка результата запроса

`fetchone()` - nри каждом вызове возвращает одну заnись из результата заnроса в виде кортежа, а затем перемешает указатель текущей nозиции.
* Если записей больше нет, метод возвращает значение `None`.

In [45]:
# import  sqlite3
con  =  sqlite3.connect("catalog.db") 
cur  =  con.cursor() 
cur.execute('SELECT * FROM Artist') 

fo = cur.fetchone() 

print(fo) 

(1, 'AC/DC', None)


`__next__()` - nри каждом вызове возвращает одну заnись из результата заnроса в виде кортежа,  а  затем  nеремешает указатель текущей  nозиции.  Если записей  больше нет, метод  возбуждает  исключение  `StopIteration`.  Выведем  все  записи  из  таблицы  user с nомощью метода `next()`.

In [46]:
# import  sqlite3
con  =  sqlite3.connect("catalog.db") 
cur  =  con.cursor() 
cur.execute('SELECT * FROM Artist') 

print(next(cur))
print(next(cur))
print(next(cur))

(1, 'AC/DC', None)
(2, 'Aerosmith', None)
(3, 'Alanis Morissette', None)


Цикл for на каждой итерации вызывает метод \_\_next\_\_()  автоматически. Поэтому для nеребора заnисей достаточно указать объект-курсор в качестве nараметра цикла.

In [47]:
# import  sqlite3
con  =  sqlite3.connect("catalog.db") 
cur  =  con.cursor() 
cur.execute('SELECT * FROM Artist') 

for v in cur:
#     fh = cur.fetchone() 
    print(v) 

(1, 'AC/DC', None)
(2, 'Aerosmith', None)
(3, 'Alanis Morissette', None)
(4, 'Alanis Morissette', 'My test comment')
(5, 'Apocalyptica', 'It is my favorite artist')
(6, "Guns N' Roses", 'Funny guys')
(7, 'Iron Maiden', 'Test symbols v " v \' v ')
(8, 'The Clash', None)
(9, 'The Cult', None)
(10, 'The Doors', None)
(11, 'The Police', None)
(12, 'The Rolling Stones', None)
(13, 'The Police', None)
(14, 'The Rolling Stones', None)
(15, 'The Tea Party', None)


In [48]:
# import  sqlite3
con  =  sqlite3.connect("catalog.db") 
cur  =  con.cursor() 
cur.execute('SELECT * FROM Artist') 

for rid, name, comment in cur:
#     fh = cur.fetchone() 
    print('rid: {:2}, name: {:25}, comment: {}'.format(rid, name, comment))

rid:  1, name: AC/DC                    , comment: None
rid:  2, name: Aerosmith                , comment: None
rid:  3, name: Alanis Morissette        , comment: None
rid:  4, name: Alanis Morissette        , comment: My test comment
rid:  5, name: Apocalyptica             , comment: It is my favorite artist
rid:  6, name: Guns N' Roses            , comment: Funny guys
rid:  7, name: Iron Maiden              , comment: Test symbols v " v ' v 
rid:  8, name: The Clash                , comment: None
rid:  9, name: The Cult                 , comment: None
rid: 10, name: The Doors                , comment: None
rid: 11, name: The Police               , comment: None
rid: 12, name: The Rolling Stones       , comment: None
rid: 13, name: The Police               , comment: None
rid: 14, name: The Rolling Stones       , comment: None
rid: 15, name: The Tea Party            , comment: None


`fetchmany([size=cursor.arraysize])` - при каждом вызове возвращает список записей из результата запроса, а затем перемещает указатель текущей nозиции.
* Каждый элемент списка является  кортежем.
* Количество элементов, выбираемых за один раз,  задается с nомощью необязательного nараметра или значения атрибута arraysize объекта-курсора.
* Если количество заnисей в результате заnроса меньше указанного количества элементов сnиска, то количество элементов сnиска будет соответствовать оставшемуся количеству заnисей.
* Если заnисей больше нет, метод возвращает пустой сnисок.

`fetchall()` - возвращает список всех (или всех оставшихся) записей из результата запроса.
* Каждый элемент списка является кортежем. 
* Если записей больше нет, то метод возвращает пустой список.

In [49]:
# import  sqlite3
con  =  sqlite3.connect("catalog.db") 
cur  =  con.cursor() 
cur.execute('SELECT * FROM Artist') 

print('cur.arraysize: ',cur.arraysize)
fm = cur.fetchmany(10) 
print(fm) 

print()
fa = cur.fetchall() 
print(fa) 

cur.arraysize:  1
[(1, 'AC/DC', None), (2, 'Aerosmith', None), (3, 'Alanis Morissette', None), (4, 'Alanis Morissette', 'My test comment'), (5, 'Apocalyptica', 'It is my favorite artist'), (6, "Guns N' Roses", 'Funny guys'), (7, 'Iron Maiden', 'Test symbols v " v \' v '), (8, 'The Clash', None), (9, 'The Cult', None), (10, 'The Doors', None)]

[(11, 'The Police', None), (12, 'The Rolling Stones', None), (13, 'The Police', None), (14, 'The Rolling Stones', None), (15, 'The Tea Party', None)]


### Управление транзакциями

Перед выполнением первого запроса автоматически запускается транзакция.  Поэтому все запросы, изменяюшие записи (INSERT,  REPIACE,  UPDATE  и DELETE),  необходимо завершать вызовом метода `commit()` объекта соединения. 
* Если метод не вызвать и  при этом закрыть соединение с  базой данных, то  все  произведенные изменения будут отменены.
* Транзакция может автоматически завершаться при выполнении запросов CREATE  ТАВLЕ, VACUUM и некоторых других. После выполнения этих запросов транзакция запускается снова. 
* Если необходимо отменить изменения, то следует вызвать метод `rollback()` объекта соединения. 

In [50]:
# import  sqlite3
con = sqlite3.connect("catalog.db") 
cur = con.cursor() #  НЕ создаем объект-курсор 
# SQL выражение 

sql = '''
INSERT INTO Artist (Name) 
VALUES (?)'''

val = ('New value',)

try:  
    cur.execute(sql, val)
    con.rollback() 
    cur.execute('SELECT * FROM Artist WHERE Name = ?', val)
    print('Результат вставки', cur.fetchall())
    
except sqlite3.DatabaseError as err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() # Завершаем транзакцию
    cur.close() # Закрываем объект-курсор 
    con.close() # Закрываем соединение

Результат вставки []
Запрос успешно выполнен


Уnравлять транзакцией можно с помощью nараметра `isolation _level` в функции `connect()`, а также с  nомощью атрибута `isolation _level`  объекта соединения. Доnустимые значения: 
* "DEFERRED"
* "IММEDIATE"
* "EXCLUSIVE"
* пустая строка 
* `None`.  

Первые три значения nередаются в инструкцию BEGIN.  Если в качестве значения указать `None`, то транзакция заnускаться не будет.  В этом случае  нет  необходимости вызывать метод `commit()`.  Все изменения будут сразу сохраняться в базе данных. 

In [51]:
sql = '''
INSERT INTO Artist (Name) 
VALUES (?)'''

val = ('New value two',)

con = sqlite3.connect("catalog.db",  isolation_level=None) 
# Другой способ установки уровня изоляции:
# con.isolation_level = None # Отключение запуска транзакции
print('con.isolation_level: ', con.isolation_level)
cur = con.cursor() 

cur.execute (sql, val) 
con.close() 

con  =  sqlite3.connect("catalog.db") 
cur = con.cursor() 

cur.execute('SELECT * FROM Artist WHERE Name = ?', val)

print(cur.fetchall())
con.close() 

con.isolation_level:  None
[(16, 'New value two', None)]


###  Обработка исключений

Модуль sqliteз поддерживает следующую иерархию исключений: 

* Exception 
    * Warning 
    * Error 
        * InterfaceError 
        * DatabaseError 
            * DataError 
            * OperationalError 
            * IntegrityError 
            * InternalError 
            * ProgrammingError 
            * NotSupportedError 
            
Базовым классом самого верхнего уровня является класс `Exception`.  Все остальные исключения  определены  в  модуле  sqliteЗ.  Поэтому при указании  исключения except  следует  предварительно  указать  названИе  модуля sqliteЗ. DatabaseError).

Исключения возбуждаются в следующих случаях: 
* warning - при наличии важных предупреждений; 
в  инструкции 
(например, 
* Error - базовый класс для всех остальных исключений, возбуждаемых в случае ошибки. Если указать этот класс в инструкции except, то будут перехватываться все ошибки: 
* InterfaceError - при  ошибках,  которые связаны  с  интерфейсом  базы  данных,  а  не с самой базой данных; 
* DatabaseError - базовый класс для исключений, которые связаны с базой данных; 
* DataError - при ошибках, возникающих при обработке данных; 
* OperationalError - вызывается  при ошибках,  которые связаны  с  операциями  в  базе данных, например, при синтаксической ошибке в SQL-зaпpoce, несоответствии количества полей в инструкции INSERT,  отсутствии поля с указанным именем и т. д.  Иногда не зависит от правильиости SQL-зaпpoca; 
* IntegrityError - при наличии проблем с внешними ключами или индексами; 
* InternalError - при внутренней ошибке в базе данных; 
* ProgrammingError - возникает при ошибках проrраммирования. Например, количество переменных, указанных во втором параметре метода execute(),  не совпадает с  количеством специальных символов в SQL-зaпpoce; 
* NotSupportedError - при использовании методов, не поддерживаемых базой данных. 

Язык Python nоддерживает nротокол  менеджеров контекста with.  Этот nротокол  гарантирует выnолнение завершающих действий вне зависимости от того,  nроизошло исключение внутри блока кода или нет.  В модуле sqlite3 объект соединения nоддерживает этот nротокол. Если внутри блока with  не nроизошло исключение, то автоматически  вызывается  метод `commit()`.  В nротивном случае  все  изменения  отменяются с  nомощью метода `rollback()`.

In [23]:
# import  sqlite3
con = sqlite3.connect("catalog.db") 
cur = con.cursor() #  НЕ создаем объект-курсор 
# SQL выражение 

sql = '''
INSERT INTO Artist (Name) 
VALUES (?)'''

val = ('New value 3',)

try:
    with con:
        con.execute(sql, val)
    
except sqlite3.DatabaseError as err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() # Завершаем транзакцию
    cur.close() # Закрываем объект-курсор 
    con.close() # Закрываем соединение

Запрос успешно выполнен


# Нормальные формы <a class="anchor" id="нормальные-формы"></a>
-
* [к оглавлению](#разделы)

Устранение дублирования информации важно по двум причинам:
− устранив дублирование, можно добиться существенной экономии
памяти;
− если некоторое значение поля повторяется несколько раз, то при
корректировке данных необходимо менять содержимое всех этих полей, в
противном случае нарушится целостность данных. 

Нормализация отношений – формальный аппарат ограничений на
формирование отношений (таблиц), который позволяет устранить дублирование, обеспечивает непротиворечивость хранимых в базе данных,
уменьшает трудозатраты на ведение (ввод, корректировку) базы данных.


При неправильно спроектированной схеме реляционной БД могут возникнуть аномалии выполнения операций модификации данных.

Аномалия обновления может возникнуть в том случае, когда информация дублируется. Другие аномалии возникают тогда, когда две и более сущности объединены в одно отношение.
Например:
* Аномалия обновления: изменился адрес поставщика. Если от него было несколько поставок, то придется менять несколько записей.
* Аномалия удаления: при удалении в архив записей обо всех поставках определённого поставщика все данные об этом поставщике (название, адрес) будут утеряны.
* Аномалия добавления: нельзя добавить сведения о поставщике, пока от него нет ни одной поставки. 
Для решения проблемы аномалии модификации данных при проектировании РБД проводится нормализация отношений.

__Первая нормальная форма__

<center>         
    <img src="./img/nf0.png" alt="Стек технологий Python для обработки данных и научных расчетов" style="width: 700px;"/>
    <b>Пример содержимого таблицы "Книги" в ненормализованном виде</b>
</center>

Первая нормальная форма

Введём понятие простого и сложного атрибута:
* __Простой атрибут__ – это атрибут, значения которого атомарны (т.е. неделимы).  Неделимость поля означает, что содержащиеся в нем значения не должны делиться на более мелкие или представлять из себя повторяющиеся группы. 
* __Сложный атрибут__ может иметь значение, представляющее собой конкатенацию нескольких значений одного или разных доменов. 

__Первая нормальная форма (1НФ)__ - отношение приведено к 1НФ, если все его атрибуты простые (каждый его кортеж содержит только одно значение для каждого из атрибутов).


__В реляционной модели отношение всегда находится в первой нормальной форме__ по определению понятия отношение. Что же касается различных таблиц, то они могут не быть правильными представлениями отношений и, соответственно, могут не находиться в 1НФ.

Для нахождения в 1НФ таблица должна удовлетворять следующим пяти условиям:
1. Нет упорядочивания строк сверху вниз (другими словами, порядок строк не несет в себе никакой информации).
2. Нет упорядочивания столбцов слева направо (другими словами, порядок столбцов не несет в себе никакой информации).
3. Нет повторяющихся строк.
4. Каждое пересечение строки и столбца содержит ровно одно значение из соответствующего домена (и больше ничего).
    * что здесь __домен__?
5. Все столбцы являются обычными

<center>         
    <img src="./img/nf1.png" alt="Приведение данных к 1НФ" style="width: 700px;"/>
    <b>Приведение таблицы "Книги" к 1НФ</b>
</center>

__Вторая нормальная форма__

__Отношение находится во 2НФ__, если:
* оно приведено к 1НФ
* каждый неключевой атрибут __функционально полно зависит__ от составного ключа.

* Понятие __функциональной зависимости__: 
    * Пусть X и Y – атрибуты (группы атрибутов) некоторого отношения. 
    * Говорят, что Y функционально зависит от X, если в любой момент времени каждому значению X=х соответствует единственное значение Y=y: X->Y (икс определяет зет).
    * При этом любому значению Y=y может соответствовать несколько значений Х=(х1, х2,…)).
    
Смысл определения в том, что если Y функционально зависит от X, то каждый из кортежей, имеющих одно и то же значение X, должен иметь также одно и то же значение Y. Значения X и Y могут изменяться время от времени, но при этом должны изменяться так,
чтобы каждое уникальное значение X имело только одно уникальное значение Y, связанное с ним.     
    
* __Полная функционально зависимость__ означает, что если потенциальный ключ является составным, то атрибут зависит от всего ключа и не зависит от его частей.
    * Если потенциальный ключ является __простым__, то есть состоит из единственного атрибута, то любая функциональная зависимость от него является полной. 
    * Если потенциальный ключ является __составным__, то, согласно определению 2НФ, в отношении __не должно быть неключевых атрибутов, зависящих от части составного потенциального ключа__.
* Вторая нормальная форма запрещает наличие неключевых атрибутов, которые вообще не зависят от потенциального ключа. Т.е. __запрещается создавать отношения как несвязанные (хаотические, случайные) наборы атрибутов__.

Приведение ко 2НФ:
* построить проекцию, исключив атрибуты, которые не находятся в функционально полной зависимости от составного ключа;
* построить дополнительные проекции на часть составного ключа и атрибуты, функционально зависящие от этой части ключа.

<center>         
    <img src="./img/nf2_1.png" alt="Приведение данных к 2НФ" style="width: 700px;"/>
    <b>Приведение таблицы "Книги" к 2НФ: отношение "Книги"</b>
</center>

<center>         
    <img src="./img/nf2_2.png" alt="Приведение данных к 2НФ" style="width: 300px;"/>
    <b>Приведение таблицы "Книги" к 2НФ: отношение "Книги-Авторы-Редакторы"</b>
</center>



Частичные функциональные зависимости (отсутствие приведения к 2НФ) приводят к __аномалиям__:

* избыточнму дублированию данных 
    * Например заголовок книги повторяется много раз для книг с несколькими авторами.
* проблеме контроля правильности данных в случае их изменения
    * Например изменение кода одной книги потребует внесения изменений в нескольких кортежах.
* проблеме добавления: нельзя добавить сведения в случае отсутсвия связанных с ним факта.
    * Например, нельзя добавить данные о поставщике, пока от него нет ни одной поставки.     
* проблема удаления записей: удаление кортежа может привести к потере сведений, относящихся не только к этому кортежу
    * Удаление записи о книге может привести к удаелнию информации о наименовании темы. 
    

__Третья нормальная форма__

__Функциональная зависимость__ множества атрибутов Z от множества атрибутов X __является транзитивной__, если существует такое множество атрибутов Y, что:
* X->Y и Y->Z
* при этом ни одно из множеств X, Y и Z не является подмножеством другого, то есть функциональные зависимости X->Z, X->Y и Y->Z не являются тривиальными
* отсутствует функциональная зависимость Y->X.

__Третья нормальная форма (3НФ)__: отношение находится в 3НФ, если:
* оно находится во 2НФ
* в нем отсутствуют транзитивные зависимости неключевых атрибутов от ключа.

Исключение: 
если для атрибутов X,Y,Z есть транзитивная зависимость X->->Z, и при этом Y->X или Z->Y, то такая зависимость не требует декомпозиции отношения. 

_Например_ :
для отношения АВТОМОБИЛИ с первичным ключом Государственный номерной знак и полями № кузова и № двигателя очевидно, что номера кузова и двигателя зависят как друг от друга, так и от первичного ключа. Но эта зависимость взаимно однозначная, поэтому декомпозиция отношения не нужна. 


<center>         
    <img src="./img/nf3_1.png" alt="Приведение данных к 2НФ" style="width: 700px;"/>
    <b>Отношения "Книги", приведенные к 3НФ</b>
</center>

<center>         
    <img src="./img/nf3_2.png" alt="Приведение данных к 2НФ" style="width: 300px;"/>
    <b>Приведение к 3НФ отношения "Книги": введение отношения "Рубрикатор"</b>
</center>

Чтобы устранить транзитивную зависимость и привести отношение в
3НФ, необходимо:
* построить проекцию без атрибутов, находящихся в транзитивной зависимости от ключа;
* построить отдельные проекции на неключевые атрибуты, между которыми имеется функциональная зависимость;
* получившиеся отношения с одинаковыми ключами соединить. 

Наличие транзитивных зависимостей порождает __аномалии__ следующего характера:
* избыточно дублирование информации
* сложное редактирование данных: изменение, требует поиска и замены его во всех кортежах где эти данные могут дублироваться;
* проблема удаления записей: удаление кортежа может привести к потере сведений о связанных категориях
* проблема добавления записей: невозможно дбавить сведения, если не добавлены связанные категории

__Анализ применимости нормализации представления данных__

Сравнение агрегированного и нормализованного представления:



<center>         
    <img src="./img/norm_form.png" alt="Приведение данных к 2НФ" style="width: 500px;"/>
    <b>Нормализованное представление</b>
</center>

<center>         
    <img src="./img/agr_form.png" alt="Приведение данных к 2НФ" style="width: 500px;"/>
    <b>Агрегированное представление</b>
</center>


__Нормализация данных__
* __Плюсы:__
    * <b class="b">+</b> Целостность даннх при обновлении (изменяем данные в одной таблице, а не в нескольких)
    * <b class="b">+</b> Ориентированность на широкий спектр запросов к данным
* __Минусы:__
    * <b class="b">-</b> Низкая скорость чтения при использовании объединений (joins)
    * <b class="b">-</b> Несоотвествие объектной модели приложения физической структуре данных
    * <b class="b">-</b> Усложенение структуры БД при высокой степени нормализации
    * <b class="b">-</b> Неэффективность в распределенной среде  

__Данные в виде агрегатов__
* __Плюсы:__
    * <b class="b">+</b> Высокая скорость чтения при отсутствии объединений (joins)
    * <b class="b">+</b> Лучший способ добиться большой скорости на чтение в распределенной среде
    * <b class="b">+</b> Возможность хранить физические объекты в том виде, в котором с ними работает приложение (легче кодировать и меньше ошибок при преобразовании)
* __Минусы:__
    * <b class="b">-</b> Оптимизация только под определенный вид запросов.
    * <b class="b">-</b> Сложность при обновлении денормализованных данных.
    
    
__Организация работы Join__

* Введение в язык SQL.ppt (слайды 20+)

TODO: Индексы

__Подход NoSQL__

* NoSQL — ряд подходов, к реализации хранилищ баз данных, имеющих существенные отличия от реляционных СУБД.
* NoSQL = Not ONLY SQL. Подход NoSQL не является отрицанием реляционного подхода (SQL), а рассматривает его как важный, но не универсальный инструмент. 

Черты, присущие подходам NoSQL (к некоторым подходам в рамках NoSQL относятся не все свойства):
* Является большим хранилищем сериализованных объектов
    * Поиск информации по ID
* В общем случае сложные запросы к данным не поддерживаются
* Не имеют структурированной (а подчас и вообще какой-либо) схемы (нет реляционной модели)
* Ориентированы на работу с денормализованными данными
* Являются готовыми решениями для создания распределенных хранилищ данных (на основе кластеров ) из-за этого не поддерживают требований ACID (Atomicity — Атомарность, Consistency — Согласованность, Isolation — Изолированность, Durability — Стойкость)
* Любой узел распределенного хранилища может отвечать на любой запрос
* Любое изменение (добавление) информации может выполнятся для любого узла хранилища и со временем распространится на другие узлы

__Документо-ориентированные базы__

Документо-ориентированные базы похожи на Key-Value базы, но отличаются тем, что БД знает, что из себя представляют значения. Обычно, значением является некоторый документ или объект, к структуре которого можно делать запросы. 

Представители (см.: https://db-engines.com/en/ranking/document+store):
* MongoDB
* Couchbase
* Firebase.


__MongoDB__ — документоориентированная  СУБД с открытым исходным кодом, не требующая описания схемы таблиц, класса NoSQL, использует JSON-подобные документы и схему базы данных. 
* Fast introduction: https://www.youtube.com/watch?v=EE8ZTQxa0AM 
* Система поддерживает ad-hoc-запросы: они могут возвращать конкретные поля документов. 
* Поддерживается JavaScript в запросах, пользовательские JavaScript-функции в функциях агрегации (аналогах map-reduce)
* Поддерживается поиск по регулярным выражениям
* База данных MongoDB распространена в системах хранения и регистрация событий в системах управления документами и контентом.
* Система может работать с набором реплик, то есть, содержать две или более копии данных на различных узлах.
* Система масштабируется горизонтально, используя sharding объектов баз данных 


* Краткое руководство по MongoDB: https://coderlessons.com/tutorials/bazy-dannykh/uchitsia-mongodb/mongodb-kratkoe-rukovodstvo
* Tutorial MongoDB на Python: https://realpython.com/introduction-to-mongodb-and-python/ 
* https://www.w3schools.com/python/python_mongodb_getstarted.asp 